In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [8]:
for n, p in model.named_parameters():
    print(n, p.shape)

model.embed_tokens.weight torch.Size([151936, 1536])
model.layers.0.self_attn.q_proj.weight torch.Size([1536, 1536])
model.layers.0.self_attn.q_proj.bias torch.Size([1536])
model.layers.0.self_attn.k_proj.weight torch.Size([256, 1536])
model.layers.0.self_attn.k_proj.bias torch.Size([256])
model.layers.0.self_attn.v_proj.weight torch.Size([256, 1536])
model.layers.0.self_attn.v_proj.bias torch.Size([256])
model.layers.0.self_attn.o_proj.weight torch.Size([1536, 1536])
model.layers.0.mlp.gate_proj.weight torch.Size([8960, 1536])
model.layers.0.mlp.up_proj.weight torch.Size([8960, 1536])
model.layers.0.mlp.down_proj.weight torch.Size([1536, 8960])
model.layers.0.input_layernorm.weight torch.Size([1536])
model.layers.0.post_attention_layernorm.weight torch.Size([1536])
model.layers.1.self_attn.q_proj.weight torch.Size([1536, 1536])
model.layers.1.self_attn.q_proj.bias torch.Size([1536])
model.layers.1.self_attn.k_proj.weight torch.Size([256, 1536])
model.layers.1.self_attn.k_proj.bias tor

In [15]:
for n, p in model.named_modules():
    print(n, p.__class__)

 <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'>
model <class 'transformers.models.qwen2.modeling_qwen2.Qwen2Model'>
model.embed_tokens <class 'torch.nn.modules.sparse.Embedding'>
model.layers <class 'torch.nn.modules.container.ModuleList'>
model.layers.0 <class 'transformers.models.qwen2.modeling_qwen2.Qwen2DecoderLayer'>
model.layers.0.self_attn <class 'transformers.models.qwen2.modeling_qwen2.Qwen2SdpaAttention'>
model.layers.0.self_attn.q_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.k_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.v_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.o_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.rotary_emb <class 'transformers.models.qwen2.modeling_qwen2.Qwen2RotaryEmbedding'>
model.layers.0.mlp <class 'transformers.models.qwen2.modeling_qwen2.Qwen2MLP'>
model.layers.0.mlp.gate_proj <class 'torch.nn.modules.linear.Linear'>
model.layer

In [16]:

prompt = "How many r's are in the word 'strawberry'?"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


The word "strawberry" contains 3 "r"s.
